# Capstone Project

## this notebook will be mainly used for the capstone project


In [3]:
#Dataframe manipulation library
import pandas as pd
import numpy as np

In [5]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Business Problem: JS hotel just opened in Manahattan NY. They are targeting out-of-town visitors who want to explore all that Manahattan has to offer. Unfortunetely, due to size limitations, the buidling does not come equiped with a gym. However, they wan't to provide health and wellness to their customers.

### Solution: Given this, the hotel wants to contact gyms in the area for partnership and to put together a packet detailing where all the gyms in the area are located. The hotel really values high customer satisfaction so they also want this ethos to be reflected in the gyms they partner with.

### Data: I will pull location data for the Manahattan burough of NY, and use the FourSquare API to find the  gyms in the area and filter to see which ones have the highest ratings. For example, I will search for gyms in the area with FourSquare's Venue API that are within a certain radius of the hotel.